In [1]:
import sys
sys.path.append('../')  # Adjust the path as necessary to reach the root of your projectmport logging

from scrape.models import *

from scrape.scrape import *

In [2]:

@dataclass
class FujiRecipeLink:
    name: str
    url: str

    recipe_url_pattern: str = r"https?://fujixweekly\.com/\d{4}/\d{2}/\d{2}/.*recipe/$"

    def is_valid_recipe_link(self) -> bool:
        # Check if the URL is None, the starting point, or the ending point
        if self.url is None or self.url == "#content" or "twitter" in self.url:
            return False

        return bool(re.match(FujiRecipeLink.recipe_url_pattern, self.url))

@dataclass
class FujiRecipes:
    sensor: FujiSensor  # Assuming FujiSensor is defined somewhere
    base_sensor_url: str
    related_recipes: list[FujiRecipe]  # Assuming FujiRecipe is defined somewhere

    @staticmethod
    def soup_representation(url: str) -> BeautifulSoup:
        page = requests.get(url, timeout=TIMEOUT_SECONDS)
        soup = BeautifulSoup(page.content, "html.parser")
        return soup

    @classmethod
    def max_recipes(cls, sensor_url) -> int:
        soup = cls.soup_representation(sensor_url)
        recipe_links = soup.find_all("a", href=re.compile(FujiRecipeLink.recipe_url_pattern))
        return len(recipe_links)

    @classmethod
    def fetch_recipes(cls, sensor, sensor_url) -> list[FujiRecipe]:
        soup = cls.soup_representation(sensor_url)
        all_links_for_sensor = soup.find_all("a")

        related_recipes = []
        for link in all_links_for_sensor:
            link_object = FujiRecipeLink(url=link.get('href'), name=link.text)
            if link_object.is_valid_recipe_link():
                sensor_recipe = FujiRecipe(sensor=sensor, link=link_object)
                if sensor_recipe in related_recipes:
                    logger.warning(f"Recipe {sensor_recipe.link.name} already fetched.")
                else:
                    related_recipes.append(sensor_recipe)

        # Validation Step
        if len(related_recipes) > cls.max_recipes(sensor_url):
            logger.warning(f"More recipes fetched ({len(related_recipes)}) than the expected maximum.")
        return related_recipes



sensor_recipes = {}
for sensor, sensor_url in GLOBAL_SENSOR_LIST.items():
    fuji_recipes_for_sensor = FujiRecipes.fetch_recipes(sensor, sensor_url)

    for recipe in fuji_recipes_for_sensor:
        logger.info('Saving recipe "%s"', recipe.link.name)
        recipe.save()


2023-11-20 13:08:11,391 - scrape.scrape - WARNING - Recipe Kodak Portra 160 v2 — Fujifilm X-T5 (X-Trans V) Film Simulation Recipe already fetched.
2023-11-20 13:08:11,391 - scrape.scrape - WARNING - Recipe Kodak Portra 160 v2 — Fujifilm X-T5 (X-Trans V) Film Simulation Recipe already fetched.
2023-11-20 13:08:11,746 - scrape.scrape - INFO - Saving recipe "Nostalgia Negative"


TemplateNotFound: ../templates/fp1.jinja2

[]